Example and Explanation of Using Historia
-----------------------------------------------------------

This notebook will walk through how to use Historia on an example Android application.
The process is roughly:
1. Choose a location and safety property in the application
2. Run Historia with no additional CBCFTL specifications
3. Look at the alarm
4. Add CBCFTL specifications to remove the alarm
5. After adding enough sound CBCFTL specifications, we can prove the example

The example we will be using is the motivating example of our paper.  The full compiled app and source code may be found in the `AntennapodPlayerFragment_fix` directory.  However, feel free to modify this notebook and run it on other open source applications.  The only external input to this notebook that is specific to this example is the APK compiled in debug mode.  The CBCFTL specifications used are written below.

Note that abstract messages, m̂ in the paper, are defined in `Specifications.scala` and are interchangeable with the Once operator (i.e. O m̂).

In [2]:
// location of the apk under analysis
val inputApk = "/home/notebooks/AntennapodPlayerFragment_fix/app/build/outputs/apk/debug/app-debug.apk"

// a few dependencies for the notebook:
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
import $ivy.`com.lihaoyi::scalatags:0.12.0`

inputApk: String = "/home/notebooks/AntennapodPlayerFragment_fix/app/build/outputs/apk/debug/app-debug.apk"

Choosing a Location and Safety Property
---------------------------------------

For reference, the code we are analyzing is printed by the cell below.

In [19]:
import better.files._

println(File("/home/notebooks/AntennapodPlayerFragment_fix/app/src/main/java/com/example/row1antennapodrxjava/ui/main/PlayerFragment.java").contentAsString
        .split('\n')
        .zipWithIndex // add line numbers
        .filter{case (line,ind) => !line.startsWith("import") && !line.trim.startsWith("//") && line.trim != ""} // remove some clutter
        .map{case (line,ind) => s"${ind + 1}  $line"}
        .mkString("\n")
       ) 

1  package com.example.row1antennapodrxjava.ui.main;
25  public class PlayerFragment extends Fragment implements Action1<Object> {
27      private Subscription sub;
29      public static PlayerFragment newInstance() {
30          return new PlayerFragment();
31      }
33      @Nullable
34      @Override
35      public View onCreateView(@NonNull LayoutInflater inflater, @Nullable ViewGroup container,
36                               @Nullable Bundle savedInstanceState) {
37          return inflater.inflate(R.layout.main_fragment, container, false);
38      }
40      @Override
41      public void onCreate(@Nullable Bundle savedInstanceState) {
42          super.onCreate(savedInstanceState);
43      }
45      @Override
46      public void onActivityCreated(@Nullable Bundle savedInstanceState) {
47          super.onActivityCreated(savedInstanceState);
48          sub = Single.create(a -> {
55          }).subscribeOn(Schedulers.newThread())
56                  .observeOn(AndroidSchedulers.m

import better.files._



The code above is a more complete version of the code in Figure 2 (a).  We would like to prove that dereferencing `act` on line 27 cannot crash.  The first step is to tell Historia what we would like to prove and where it is.  This is done by creating a `InitialQuery`, Specifically a `ReceiverNonNull`.

In [44]:
import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,ReceiverNonNull}
import edu.colorado.plv.bounder.lifestate.LifeState

// The method signature unambiguously identifies the method in the application
val methodSignature = LifeState.Signature("com.example.row1antennapodrxjava.ui.main.PlayerFragment",
          "void call(java.lang.Object)")

val initialQuery = ReceiverNonNull(
        methodSignature,
        63, // line number in source code file
        None //Some(".*toString.*") // regular expression matching receiver (in case multiple dereferences on one line)
    )

import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,ReceiverNonNull}

import edu.colorado.plv.bounder.lifestate.LifeState

// The method signature unambiguously identifies the method in the application

methodSignature: LifeState.Signature = Signature(
  "com.example.row1antennapodrxjava.ui.main.PlayerFragment",
  "void call(java.lang.Object)"
)
initialQuery: ReceiverNonNull = ReceiverNonNull(
  Signature(
    "com.example.row1antennapodrxjava.ui.main.PlayerFragment",
    "void call(java.lang.Object)"
  ),
  63,
  None
)

Run Historia With No Additional CBCFTL Specifications
-----------------------------------------------------

Next we run historia with no constraints on what the framework may do.  In many cases, an app can be proven safe before adding CBCFTL specifications.  If it cannot, the counter example is useful for writing the CBCFTL.

In [51]:
// load historia code

val jarpath = s"/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
interp.load.cp(os.Path(jarpath))

import edu.colorado.plv.bounder.{Driver,RunConfig, BounderUtil} // Historia utilities
import upickle.default.read
import upickle.default.write

// define a function to call the JAR implementation of Historia with a configuration
// If changes are made to Historia, run "sbt compile" in the /home/implementation directory to regenerate the Historia JAR

def runHistoriaWithSpec(configPath:File):Driver.LocResult = {
    val javaMemLimit=8 // Gb Note that this only limits JVM not JNI which can go significantly higher
    val historiaJar = "/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
    val apkRootDir = "/home/historia_generalizability"
    val outDir = configPath.parent.toString
    val config = read[RunConfig](configPath.contentAsString)
    val outSubdir = config.outFolder.get.replace("${baseDirOut}",outDir)
    val cmd = s"java -Xmx${javaMemLimit}G -jar ${historiaJar} -m verify -c ${configPath} -b ${apkRootDir} -u ${outDir} -o MEM --debug"
    BounderUtil.runCmdStdout(cmd)

    read[Driver.LocResult]((outSubdir / "result_0.txt").contentAsString)
}



jarpath: String = "/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import edu.colorado.plv.bounder.{Driver,RunConfig, BounderUtil} // Historia utilities

import upickle.default.read

import upickle.default.write

// define a function to call the JAR implementation of Historia with a configuration
// If changes are made to Historia, run "sbt compile" in the /home/implementation directory to regenerate the Historia JAR


defined function runHistoriaWithSpec

Next we create the `RunConfig` that specifies things like the APK, the output folder, the initial query, and the CBCFTL.  This is all written to a `.json` file used later.

In [52]:
val outputDir = File("/home/notebooks/ExampleOut")
val cfg =  RunConfig(apkPath = inputApk.toString, 
          outFolder = Some(outputDir.toString),
          initialQuery = List(initialQuery), truncateOut=false)

val cfgPath = (outputDir / "cfg.json")
cfgPath.overwrite(write(cfg))  

outputDir: File = /home/notebooks/ExampleOut
cfg: RunConfig = RunConfig(
  "/home/notebooks/AntennapodPlayerFragment_fix/app/build/outputs/apk/debug/app-debug.apk",
  Some("/home/notebooks/ExampleOut"),
  None,
  TopSpecSet,
  List(
    ReceiverNonNull(
      Signature(
        "com.example.row1antennapodrxjava.ui.main.PlayerFragment",
        "void call(java.lang.Object)"
      ),
      63,
      None
    )
  ),
  -1,
  5,
  ExpTag("", "", ""),
  600,
  false,
  None
)
cfgPath: File = /home/notebooks/ExampleOut/cfg.json
res51_3: File = /home/notebooks/ExampleOut/cfg.json

Now we can call Historia.

In [47]:
runHistoriaWithSpec(cfgPath)

res46: Driver.LocResult = LocResult(
  ReceiverNonNull(
    Signature(
      "com.example.row1antennapodrxjava.ui.main.PlayerFragment",
      "void call(java.lang.Object)"
    ),
    63,
    None
  ),
  -1,
  AppLoc(
    SerializedIRMethodLoc(
      "com.example.row1antennapodrxjava.ui.main.PlayerFragment",
      "void call(java.lang.Object)",
      List(
        Some(
          LocalWrapper(
            "@this",
            "com.example.row1antennapodrxjava.ui.main.PlayerFragment"
          )
        ),
        Some(LocalWrapper("@parameter0", "java.lang.Object"))
      )
    ),
    SerializedIRLineLoc(
      63,
      "line: 63 virtualinvoke $r2.<java.lang.Object: java.lang.String toString()>()",
      765356857
    ),
    true
  ),
  Witnessed,
  MultiCallback,
  1L,
  DepthResult(9, 1, 1, Witnessed),
  List(
    List(
      "witnessed",
...

Look at the Alarm
-----------------
Next we look at the alarm that was found.  In this case, it finds the initial state just before the `call` callback.  The witness also shows that `getActivity` was invoked, this is important because it is where a null value may come from.

In [48]:
println(File("/home/notebooks/ExampleOut/wit.witnesses").contentAsString)

witnessed
    WITNESSED: [CB Inv] com.example.row1antennapodrxjava.ui.main.PlayerFragment void call(java.lang.Object)
       state: (    heap:           pure:          types: List()    trace: )
    pre-line: 62 $r2 = virtualinvoke r0.<com.example.row1antennapodrxjava.ui.main.PlayerFragment: androidx.fragment.app.FragmentActivity getActivity()>()
       state: ( locals:          heap:           pure:          types: List()    trace: )
    [CI Inv merge] androidx.fragment.app.FragmentActivity getActivity()
       state: ([CI Ret] androidx.fragment.app.Fragment androidx.fragment.app.FragmentActivity getActivity() locals:       locals:          heap:           pure:          types: List()    trace: )
    [CI Ret merge] androidx.fragment.app.FragmentActivity getActivity()
       state: ([CI Ret] androidx.fragment.app.Fragment androidx.fragment.app.FragmentActivity getActivity() locals:       locals:          heap:           pure:          types: List()    trace: )



Add CBCFTL Specifications to Remove the Alarm
--------------------------

Writing a CBCFTL specification consists of looking at counter examples like the one above and explaining when the framework can *NOT* do something like return a `null` value.  An easy, yet unsound, CBCFTL specification is to say `getActivity` cannot return null ever.  However, writing this specification is a useful step to show how CBCFTL specifications are constructed.  We will write a sound specification later.

We write that as `null = cb a.getActivity() -[]-> false` in the paper.  Below we show how to write this same thing for the implementation.

In [54]:

import edu.colorado.plv.bounder.lifestate.LifeState.{LSSpec, LSFalse,LSConstraint}
import edu.colorado.plv.bounder.symbolicexecutor.state.{NamedPureVar,NullVal,Equals}
import edu.colorado.plv.bounder.lifestate.SpecSignatures

val a = NamedPureVar("a")  // variables used in spec
val f = NamedPureVar("f")

val getActivityNullUnsound = LSSpec(a::f::Nil, Nil,
    LSFalse, 
    SpecSignatures.Fragment_get_activity_exit, // abstract message f = cb a.getActivity()  (defined in Specifications.scala)
    Set(LSConstraint(a, Equals, NullVal)))

import edu.colorado.plv.bounder.lifestate.LifeState.{LSSpec, LSFalse,LSConstraint}

import edu.colorado.plv.bounder.symbolicexecutor.state.{NamedPureVar,NullVal,Equals}

import edu.colorado.plv.bounder.lifestate.SpecSignatures


a: NamedPureVar = NamedPureVar("a")
f: NamedPureVar = NamedPureVar("f")
getActivityNullUnsound: LSSpec = LSSpec(
  List(NamedPureVar("a"), NamedPureVar("f")),
  List(),
  False,
  OAbsMsg(
    CIExit,
    SubClassMatcher(
      Set(
        "android.app.Fragment",
        "androidx.fragment.app.Fragment",
        "android.support.v4.app.Fragment"
      ),
      ".*Activity getActivity\\(\\)",
      "Fragment_getActivity"
    ),
    List(NamedPureVar("a"), NamedPureVar("f"))
  ),
  Set(LSConstraint(NamedPureVar("a"),  == , NULL))
)

Next we copy the configuration file with the new CBCFTL specification and re-run Historia.

In [57]:
import edu.colorado.plv.bounder.PickleSpec

val cfgWithUnsoundSpec = cfg.copy(specSet = PickleSpec(Set(getActivityNullUnsound)))
cfgPath.overwrite(write(cfgWithUnsoundSpec))
val res = runHistoriaWithSpec(cfgPath)
println(s"result: ${res.resultSummary}")

result: Proven


import edu.colorado.plv.bounder.PickleSpec


cfgWithUnsoundSpec: RunConfig = RunConfig(
  "/home/notebooks/AntennapodPlayerFragment_fix/app/build/outputs/apk/debug/app-debug.apk",
  Some("/home/notebooks/ExampleOut"),
  None,
  PickleSpec(
    Set(
      LSSpec(
        List(NamedPureVar("a"), NamedPureVar("f")),
        List(),
        False,
        OAbsMsg(
          CIExit,
          SubClassMatcher(
            Set(
              "android.app.Fragment",
              "androidx.fragment.app.Fragment",
              "android.support.v4.app.Fragment"
            ),
            ".*Activity getActivity\\(\\)",
            "Fragment_getActivity"
          ),
          List(NamedPureVar("a"), NamedPureVar("f"))
        ),
        Set(LSConstraint(NamedPureVar("a"),  == , NULL))
      )
    ),
    Set(),
    Set()
  ),
  List(
    ReceiverNonNull(
      Signature(
        "com.example.row1antennapodrxjava.ui.main.PlayerFragment",
        "void call(java.lang.Object)"
      ),
      63,
  

Note how the previous result says `proven`.  This would be a sound proof if `getActivity` could never return null. However, `getActivity` may return null if it is called after `onDestroy` or before `onCreate`.  We can update the spec to capture this behavior.

`null = cb a.getActivity() -[]-> (cb a.onCreate() NS cb a.onDestroy()) \/ HN cb a.onCreate()`

This history implication is now sound.

In [58]:
import edu.colorado.plv.bounder.lifestate.LifeState.{Not,Or,NS}

val fragmentActivityNotAttached =
    Or(NS(SpecSignatures.Fragment_onDestroy_exit, SpecSignatures.Fragment_onActivityCreated_entry), // note that NS arguments are reversed from the paper
      Not(SpecSignatures.Fragment_onActivityCreated_entry)) // "Not" can only be applied to a single message and maintain encoding in extended EPR fragment of logic

val getActivityNull = LSSpec(a::f::Nil, Nil,
    fragmentActivityNotAttached, 
    SpecSignatures.Fragment_get_activity_exit, // abstract message f = cb a.getActivity()  (defined in Specifications.scala)
    Set(LSConstraint(a, Equals, NullVal)))

import edu.colorado.plv.bounder.lifestate.LifeState.{Not,Or,NS}


fragmentActivityNotAttached: Or = Or(
  NS(
    OAbsMsg(
      CBExit,
      SubClassMatcher(
        Set(
          "android.app.Fragment",
          "androidx.fragment.app.Fragment",
          "android.support.v4.app.Fragment"
        ),
        "void onDestroy\\(\\)",
        "Fragment_onDestroy"
      ),
      List(_T_, NamedPureVar("f"))
    ),
    OAbsMsg(
      CBEnter,
      SubClassMatcher(
        Set(
          "android.app.Fragment",
          "androidx.fragment.app.Fragment",
          "android.support.v4.app.Fragment"
        ),
        "void onActivityCreated\\(android.os.Bundle\\)",
        "Fragment_onActivityCreated"
      ),
      List(_T_, NamedPureVar("f"))
    )
  ),
  Not(
    OAbsMsg(
      CBEnter,
      SubClassMatcher(
        Set(
          "android.app.Fragment",
          "androidx.fragment.app.Fragment",
          "android.support.v4.app.Fragment"
        ),
        "void onActivityCreated\

Similarly, we may write History Implication 1 from the overview.

In [59]:
  val l = NamedPureVar("l")
  val s = NamedPureVar("s")

  val subUnsub = NS(
    SpecSignatures.RxJava_subscribe_exit,
    SpecSignatures.RxJava_unsubscribe_exit)
  val callSpec:LSSpec = LSSpec(l::Nil, s::Nil, subUnsub, SpecSignatures.RxJava_call_entry)


l: NamedPureVar = NamedPureVar("l")
s: NamedPureVar = NamedPureVar("s")
subUnsub: NS = NS(
  OAbsMsg(
    CIExit,
    SubClassMatcher(
      Set("rx.Single"),
      "rx.Subscription subscribe\\(rx.functions.Action1\\)",
      "RxJava_subscribe"
    ),
    List(NamedPureVar("s"), _T_, NamedPureVar("l"))
  ),
  OAbsMsg(
    CIExit,
    SubClassMatcher(
      Set(
        "rx.Subscriber",
        "rx.SingleSubscriber",
        "rx.Subscription",
        "rx.subscriptions.RefCountSubscription"
      ),
      "void unsubscribe\\(\\)",
      "rxJava_unsubscribe"
    ),
    List(_T_, NamedPureVar("s"))
  )
)
callSpec: LSSpec = LSSpec(
  List(NamedPureVar("l")),
  List(NamedPureVar("s")),
  NS(
    OAbsMsg(
      CIExit,
      SubClassMatcher(
        Set("rx.Single"),
        "rx.Subscription subscribe\\(rx.functions.Action1\\)",
        "RxJava_subscribe"
      ),
      List(NamedPureVar("s"), _T_, NamedPureVar("l"))
    ),
    OAbsMsg(
      CIExit,
      SubClassMatcher(
        Set(
     

Let us run it again with the proper `getActivity` and `call` History Implications and see what happens.

In [60]:
val cfgWithSoundSpec = cfg.copy(specSet = PickleSpec(Set(getActivityNull,callSpec)))
cfgPath.overwrite(write(cfgWithSoundSpec))
val res = runHistoriaWithSpec(cfgPath)
println(s"result: ${res.resultSummary}")


result: Witnessed


cfgWithSoundSpec: RunConfig = RunConfig(
  "/home/notebooks/AntennapodPlayerFragment_fix/app/build/outputs/apk/debug/app-debug.apk",
  Some("/home/notebooks/ExampleOut"),
  None,
  PickleSpec(
    Set(
      LSSpec(
        List(NamedPureVar("a"), NamedPureVar("f")),
        List(),
        Or(
          NS(
            OAbsMsg(
              CBExit,
              SubClassMatcher(
                Set(
                  "android.app.Fragment",
                  "androidx.fragment.app.Fragment",
                  "android.support.v4.app.Fragment"
                ),
                "void onDestroy\\(\\)",
                "Fragment_onDestroy"
              ),
              List(_T_, NamedPureVar("f"))
            ),
            OAbsMsg(
              CBEnter,
              SubClassMatcher(
                Set(
                  "android.app.Fragment",
                  "androidx.fragment.app.Fragment",
                  "android.support.v4.app.Fragment"
                ),
                "

Another alarm occurred.  Lets look at the witness:

In [61]:
println(File("/home/notebooks/ExampleOut/wit.witnesses").contentAsString)

witnessed
    WITNESSED: [CB Inv] com.example.row1antennapodrxjava.ui.main.PlayerFragment void onActivityCreated(android.os.Bundle)
       state: (    heap:           pure: p-2  !=  NULL && p-4  !=  NULL && p-3  !=  NULL && p-1  !=  NULL         types: List(p-1:{8107,8119}, p-3:{759,6271,6272,6274,6287 ...}, p-4:{8107,8119}, p-2:{8107,8119}, p-5:{759,6271,6272,6274,6287 ...})    trace: O(CBEnter I_CBEnter_FragmentonActivityCreated ( _T_,p-4 );O(CIExit I_CIExit_RxJavasubscribe ( p-5,_T_,p-4 );O(CBEnter I_CBEnter_FragmentonActivityCreated ( _T_,p-2 );O(CIExit I_CIExit_RxJavasubscribe ( p-3,_T_,p-2 );O(CIExit I_CIExit_rxJavaunsubscribe ( _T_,p-3 );O(CBExit I_CBExit_FragmentonDestroy ( _T_,p-2 );O(CBEnter I_CBEnter_rxJavacall ( _T_,p-1 );O(CIExit I_CIExit_FragmentgetActivity ( NULL,p-1 ))
    pre-line: -1 r0 := @this: com.example.row1antennapodrxjava.ui.main.PlayerFragment
       state: ( locals: @this -> p-4         heap:           pure: p-2  !=  NULL && p-4  !=  NULL && p-3  !=  NULL && 

It can be kinda difficult to see what happened since the counter example is rather busy.  However, `onActivityCreated`  was called twice in a row in this counter example (note that we shorten `onActivityCreated` to `onCreate` in the paper).  If the framework could actually invoke this callback twice, it would start two background tasks and only one would be unsubscribed.  Therefore this alarm is precise if no further CBCFTL specifications could be written.  However, it is sound to say that `onActivityCreated` is only invoked once with the following history implication.

The abstract counter example shows each transition and the associated pre-state.
Read from the top down, they correspond to a potential execution in the app.
Practically, these are all just disjunctions at the "framework" location in the paper, they are just strung together for ease of use.

The pre-state consists of the trace (e.g. `trace: O(CBEnter I_CBEnter_rxJavacall ( _T_,p-1 );O(CIExit I_CIExit_FragmentgetActivity ( NULL,p-1 ))`), separation logic formula, (e.g. `@this->p1`), pure constraints (e.g. `p1 != NULL`), and points-to information (e.g. `types: List(p-1:{8107,8119})`).

Traces have the same meaning as the paper, they are just printed with `;` instead of `->>` and the `okhist` is elided.  The meaning of the separation logic formula is standard.  The points to analysis maintains a set of allocation sites for each pure variable represented numerically for efficiency.  There is also a call string, but it isn't printed here.

In [64]:
val Fragment_activityCreatedOnlyFirst:LSSpec = LSSpec(f::Nil, Nil,
      Not(SpecSignatures.Fragment_onActivityCreated_entry),
    SpecSignatures.Fragment_onActivityCreated_entry)


Fragment_activityCreatedOnlyFirst: LSSpec = LSSpec(
  List(NamedPureVar("f")),
  List(),
  Not(
    OAbsMsg(
      CBEnter,
      SubClassMatcher(
        Set(
          "android.app.Fragment",
          "androidx.fragment.app.Fragment",
          "android.support.v4.app.Fragment"
        ),
        "void onActivityCreated\\(android.os.Bundle\\)",
        "Fragment_onActivityCreated"
      ),
      List(_T_, NamedPureVar("f"))
    )
  ),
  OAbsMsg(
    CBEnter,
    SubClassMatcher(
      Set(
        "android.app.Fragment",
        "androidx.fragment.app.Fragment",
        "android.support.v4.app.Fragment"
      ),
      "void onActivityCreated\\(android.os.Bundle\\)",
      "Fragment_onActivityCreated"
    ),
    List(_T_, NamedPureVar("f"))
  ),
  Set()
)

After Adding Enough Sound CBCFTL Specifications, We Can Prove the Example
-------------------------------------------

Below, we create another run configuration with 3 History Implications written above and run it again with Historia.  This time the output is proven.  This 3 corresponds to the "specs" column of Table 1.

In [63]:
val cfgWithSoundSpec = cfg.copy(specSet = PickleSpec(Set(getActivityNull,callSpec, Fragment_activityCreatedOnlyFirst)))
cfgPath.overwrite(write(cfgWithSoundSpec))
val res = runHistoriaWithSpec(cfgPath)
println(s"result: ${res.resultSummary}")

result: Proven


cfgWithSoundSpec: RunConfig = RunConfig(
  "/home/notebooks/AntennapodPlayerFragment_fix/app/build/outputs/apk/debug/app-debug.apk",
  Some("/home/notebooks/ExampleOut"),
  None,
  PickleSpec(
    Set(
      LSSpec(
        List(NamedPureVar("a"), NamedPureVar("f")),
        List(),
        Or(
          NS(
            OAbsMsg(
              CBExit,
              SubClassMatcher(
                Set(
                  "android.app.Fragment",
                  "androidx.fragment.app.Fragment",
                  "android.support.v4.app.Fragment"
                ),
                "void onDestroy\\(\\)",
                "Fragment_onDestroy"
              ),
              List(_T_, NamedPureVar("f"))
            ),
            OAbsMsg(
              CBEnter,
              SubClassMatcher(
                Set(
                  "android.app.Fragment",
                  "androidx.fragment.app.Fragment",
                  "android.support.v4.app.Fragment"
                ),
                "